In [3]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [4]:
df = pd.read_csv('kiva_loans_20181016.csv')
df.head()

,id,date,activity,sector,use,funded_amount,loan_amount,diff_funded_loan,status,country_code,country,currency,gender,borrower_genders,lender_count,term_in_months,repayment_interval,tags
0,653051,1/1/14,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",300,300,0,1,PK,Pakistan,PKR,female,female,12,12,irregular,NaN
1,653053,1/1/14,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,575,575,0,1,PK,Pakistan,PKR,group,"female, female",14,11,irregular,NaN
2,653068,1/1/14,Transportation,Transportation,To repair their old cycle-van and buy another ...,150,150,0,1,IN,India,INR,female,female,6,43,bullet,"user_favorite, user_favorite"
3,653063,1/1/14,Embroidery,Arts,to purchase an embroidery machine and a variet...,200,200,0,1,PK,Pakistan,PKR,female,female,8,11,irregular,NaN
4,653084,1/1/14,Milk Sales,Food,to purchase one buffalo.,400,400,0,1,PK,Pakistan,PKR,female,female,16,14,monthly,NaN


In [5]:
df.shape

(671205, 18)

In [6]:
df.status.value_counts()

1    622877
0     48328
Name: status, dtype: int64

In [7]:
df.dtypes

id                     int64
date                  object
activity              object
sector                object
use                   object
funded_amount          int64
loan_amount            int64
diff_funded_loan       int64
status                 int64
country_code          object
country               object
currency              object
gender                object
borrower_genders      object
lender_count           int64
term_in_months         int64
repayment_interval    object
tags                  object
dtype: object

In [8]:
df.isnull().sum()

id                         0
date                       0
activity                   0
sector                     0
use                     4232
funded_amount              0
loan_amount                0
diff_funded_loan           0
status                     0
country_code               8
country                    0
currency                   0
gender                  4221
borrower_genders        4221
lender_count               0
term_in_months             0
repayment_interval         0
tags                  171416
dtype: int64

In [9]:
df1 = df[['status','funded_amount', 'loan_amount', 'activity', 'sector',  'country',
         'currency','gender','term_in_months']]

In [10]:
df1.head(2)

,status,funded_amount,loan_amount,activity,sector,country,currency,gender,term_in_months
0,1,300,300,Fruits & Vegetables,Food,Pakistan,PKR,female,12
1,1,575,575,Rickshaw,Transportation,Pakistan,PKR,group,11


In [11]:
df2 = df1.dropna()
df2 = df2.drop(['term_in_months', 'currency'], axis=1)
df2.head()

,status,funded_amount,loan_amount,activity,sector,country,gender
0,1,300,300,Fruits & Vegetables,Food,Pakistan,female
1,1,575,575,Rickshaw,Transportation,Pakistan,group
2,1,150,150,Transportation,Transportation,India,female
3,1,200,200,Embroidery,Arts,Pakistan,female
4,1,400,400,Milk Sales,Food,Pakistan,female


In [12]:
df2.shape

(666984, 7)

In [13]:
# Use Pandas get_dummies to convert categorical data

df2 = pd.get_dummies(df2)
df2.head()

,status,funded_amount,loan_amount,activity_Adult Care,activity_Agriculture,activity_Air Conditioning,activity_Animal Sales,activity_Aquaculture,activity_Arts,activity_Auto Repair,...,country_United States,country_Vanuatu,country_Vietnam,country_Virgin Islands,country_Yemen,country_Zambia,country_Zimbabwe,gender_female,gender_group,gender_male
0,1,300,300,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,575,575,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,150,150,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,200,200,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,400,400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
df2.shape

(666984, 271)

In [15]:
X = df2.drop(['status', 'loan_amount', 'funded_amount'], axis=1)
y = df2['status']

In [16]:
ss = StandardScaler()
lr = LogisticRegression()
lr_pipe = Pipeline([('sscale', ss), ('logreg', lr)])

In [17]:
lr_pipe.fit(X, y)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('sscale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [18]:
lr_pipe.score(X,y)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9285485109088074

# Divide the dataset into separate training (80% of the data) and test (20% of the data) datasets.

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# Chain the StandardScaler and Logistic Regression objects in a pipeline.

In [20]:
lr_pipe.fit(X_train, y_train)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('sscale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [21]:
lr_pipe.score(X_test, y_test)  # prediction accuracy score

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.928454163137102

In [22]:
lr_pipe.score(X_train, y_train)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


0.9284933853336007

In [23]:
y_pred = lr_pipe.predict(X_test)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [24]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [25]:
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro")) 

0.5016300475066449
0.7019798334007088
0.509654404447844


# Alternative way of executing the Lograthmic Model. Lograthmic models don't require scaling.

In [26]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [27]:
print(f"Training Data Score: {logmodel.score(X_train, y_train)}")
print(f"Testing Data Score: {logmodel.score(X_test, y_test)}")

Training Data Score: 0.9285252451802611
Testing Data Score: 0.9284991416598574


In [28]:
predictions = logmodel.predict(X_test)

# 1 - Logistic Model Score

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.48      0.02      0.04      9522
           1       0.93      1.00      0.96    123875

   micro avg       0.93      0.93      0.93    133397
   macro avg       0.71      0.51      0.50    133397
weighted avg       0.90      0.93      0.90    133397



In [30]:
df4 = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

In [31]:
df4.head(10)

,Prediction,Actual
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
5,1,1
6,1,1
7,1,1
8,1,1
9,1,1


# Statistical Testing of the model for significance of independedent variables 

In [32]:
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm # conda install statsmodels - if there is an error
from scipy import stats



X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                 status   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     195.3
Date:                Sat, 20 Oct 2018   Prob (F-statistic):               0.00
Time:                        12:16:35   Log-Likelihood:                -9967.8
No. Observations:              533587   AIC:                         2.044e+04
Df Residuals:                  533336   BIC:                         2.325e+04
Df Model:                         250                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [33]:
X2 = sm.add_constant(X_test)
est = sm.OLS(y_test, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                 status   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     50.06
Date:                Sat, 20 Oct 2018   Prob (F-statistic):               0.00
Time:                        12:16:38   Log-Likelihood:                -2492.5
No. Observations:              133397   AIC:                             5469.
Df Residuals:                  133155   BIC:                             7841.
Df Model:                         241                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

/anaconda3/envs/PythonData/lib/python3.6/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/anaconda3/envs/PythonData/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [34]:
df_activity = df[['status', 'activity']]
df_activity = df_activity.dropna()
df_activity.head()

,status,activity
0,1,Fruits & Vegetables
1,1,Rickshaw
2,1,Transportation
3,1,Embroidery
4,1,Milk Sales


In [35]:
df_activity.shape

(671205, 2)

In [36]:
df_activity = pd.get_dummies(df_activity)
df_activity.head()

,status,activity_Adult Care,activity_Agriculture,activity_Air Conditioning,activity_Animal Sales,activity_Aquaculture,activity_Arts,activity_Auto Repair,activity_Bakery,activity_Balut-Making,...,activity_Utilities,activity_Vehicle,activity_Vehicle Repairs,activity_Veterinary Sales,activity_Waste Management,activity_Water Distribution,activity_Weaving,activity_Wedding Expenses,activity_Well digging,activity_Wholesale
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
X = df_activity.drop(['status'], axis=1)
y = df_activity['status']

In [38]:
lm = LogisticRegression()
lm.fit(X,y)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X)

params = np.round(params,4)

myDF3 = pd.DataFrame()
index = [0]
params = np.delete(params, index)

myDF3["Activity_Feature_Name"],myDF3["Activity_Coefficients"] = [X.columns,params]
print(myDF3)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


             Activity_Feature_Name  Activity_Coefficients
0              activity_Adult Care                 0.0798
1             activity_Agriculture                -0.5987
2        activity_Air Conditioning                -0.0285
3            activity_Animal Sales                -0.2070
4             activity_Aquaculture                -0.5387
5                    activity_Arts                 0.5732
6             activity_Auto Repair                -1.1636
7                  activity_Bakery                -0.0646
8            activity_Balut-Making                 1.1045
9             activity_Barber Shop                -0.9150
10           activity_Beauty Salon                -0.6690
11             activity_Beekeeping                -0.0949
12              activity_Beverages                -0.6162
13         activity_Bicycle Repair                 1.6725
14          activity_Bicycle Sales                 1.0377
15             activity_Blacksmith                 1.6599
16            

In [88]:
keys = [i.replace('activity_', '') for i in df_activity.columns[1:]]
activity_features = dict(zip(keys, myDF3.Activity_Coefficients.values))

activity_coef = [activity_features[i] for i in df.activity.values]
df['activity_coef'] = activity_coef
df

,id,date,activity,sector,use,funded_amount,loan_amount,diff_funded_loan,status,country_code,country,currency,gender,borrower_genders,lender_count,term_in_months,repayment_interval,tags,activity_coef
0,653051,1/1/14,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",300,300,0,1,PK,Pakistan,PKR,female,female,12,12,irregular,NaN,0.1499
1,653053,1/1/14,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,575,575,0,1,PK,Pakistan,PKR,group,"female, female",14,11,irregular,NaN,-0.6277
2,653068,1/1/14,Transportation,Transportation,To repair their old cycle-van and buy another ...,150,150,0,1,IN,India,INR,female,female,6,43,bullet,"user_favorite, user_favorite",-0.8138
3,653063,1/1/14,Embroidery,Arts,to purchase an embroidery machine and a variet...,200,200,0,1,PK,Pakistan,PKR,female,female,8,11,irregular,NaN,0.6553
4,653084,1/1/14,Milk Sales,Food,to purchase one buffalo.,400,400,0,1,PK,Pakistan,PKR,female,female,16,14,monthly,NaN,0.2808
5,1080148,1/1/14,Services,Services,purchase leather for my business using ksh 20000.,250,250,0,1,KE,Kenya,KES,female,female,6,4,irregular,NaN,-1.1439
6,653067,1/1/14,Dairy,Agriculture,To purchase a dairy cow and start a milk produ...,200,200,0,1,IN,India,INR,female,female,8,43,bullet,"user_favorite, user_favorite",0.0652
7,653078,1/1/14,Beauty Salon,Services,to buy more hair and skin care products.,400,400,0,1,PK,Pakistan,PKR,female,female,8,14,monthly,"#Elderly, #Woman Owned Biz",-0.6690
8,653082,1/1/14,Manufacturing,Manufacturing,"to purchase leather, plastic soles and heels i...",475,475,0,1,PK,Pakistan,PKR,female,female,19,14,monthly,user_favorite,2.2688
9,653048,1/1/14,Food Production/Sales,Food,"to buy a stall, gram flour, ketchup, and coal ...",625,625,0,1,PK,Pakistan,PKR,female,female,24,11,irregular,NaN,-0.1404
